In [5]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

### Preparar datos y definir features

In [6]:
def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

### Tracking con MLflow

Configuraremos MLflow para:

+ Definir un experimento.
+ Iniciar un run.
+ Registrar parámetros, métricas y modelo.
+ Visualizar resultados en la UI.

In [7]:
import mlflow

In [8]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"Run finalizado. RMSE = {rmse:.4f}")

2025/10/21 20:36:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/21 20:36:19 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/10/21 20:36:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/21 20:36:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/21 20:36:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run finalizado. RMSE = 8.9926


### Model register example

In [9]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle

In [10]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

<Experiment: artifact_location='/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/Notebooks/mlruns/2', creation_time=1761100169288, experiment_id='2', last_update_time=1761100169288, lifecycle_stage='active', name='nyc-taxi-model-registry-example', tags={}>

In [11]:
mlflow.sklearn.autolog()

In [12]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

### Nested Runs

In [13]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [14]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params) # descomprimir
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/21 20:36:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


mkdir: models: File exists


2025/10/21 20:36:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


mkdir: models: File exists


/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/21 20:36:50 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


mkdir: models: File exists


### Resgitrar modelos

1. `mlflow.<framework>.log_model()`

In [15]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


2025/10/21 20:37:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/21 20:37:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/21 20:37:16 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/21 20:37:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/21 20:37:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/21 20:37:18 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/21 20:37:18 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INF

2. `mlflow.register_model()`

In [16]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/21 20:37:28 WARNING mlflow.tracking._model_registry.fluent: Run with id 0780448a0c584eedaa5429e67257599e has no artifacts at artifact path 'model', registering model based on models:/m-51d91393d7f84abba45e1dd76e31436c instead
Created version '3' of model 'nyc-taxi-model'.


3. `MlflowClient`

In [17]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [19]:
client.create_registered_model(name="nyc-taxi-model-2")

<RegisteredModel: aliases={}, creation_timestamp=1761100664261, deployment_job_id=None, deployment_job_state=None, description=None, last_updated_timestamp=1761100664261, latest_versions=[], name='nyc-taxi-model-2', tags={}>

In [20]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

In [25]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=4
)

# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=4
)

In [26]:
# get a model version by alias
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['challenger', 'champion'], creation_timestamp=1761100667964, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1761100667964, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='0780448a0c584eedaa5429e67257599e', run_link=None, source='runs:/0780448a0c584eedaa5429e67257599e/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [27]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [28]:
# actualizar version de un modelo
client.update_model_version(
    name="nyc-taxi-model",
    version=1,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=[], creation_timestamp=1761100204383, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1761100832240, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='0780448a0c584eedaa5429e67257599e', run_link=None, source='models:/m-51d91393d7f84abba45e1dd76e31436c', status='READY', status_message=None, tags={}, user_id=None, version=1>

### Obteniendo modelos del registro de modelos

1. Obtener el modelo por versión del modelo

In [30]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 4

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

MlflowException: Model does not have the "python_function" flavor

2. Obtener el modelo por alias

In [31]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

MlflowException: Model does not have the "python_function" flavor

### Comparación de versiones y selección del nuevo modelo `champion`